In [ ]:
%matplotlib inline

import captest as pvc
from bokeh.io import output_notebook, show

#if working offline with the CapData.plot() method may fail
#run 'export BOKEH_RESOURCES=inline' at the command line before
#running the jupyter notebook

output_notebook()

# Loading Measured and Simulated Data

CapData objects hold simulated data from PVsyst and measured data from a DAS or SCADA system and provide methods to load, organize, and visualize the data.

The load_data method by default attempts to infer the type of measurement and the type of sensor in each column.  For example, a measurement from a ambient temperature sensor would be categorized as 'temp-amb-'.  A python dictionary which translates from the column titles in the input file and the inferred type is created and saved to the CapData.trans attribute. This translation dictionary is used in the CapData.view and CapData.rview methods to easily access columns of data of a certain type without renaming columns.  This dictionary is used by methods of CapTest objects to aggregate sensors of the same type.

User input is required to set which inferred measurement type is the correct one for the regression variables. The CapData.set_reg_trans method is used to set this relationship.

In [ ]:
das = pvc.CapData()

In [ ]:
das.load_data(fname='example_meas_data.csv', source='AlsoEnergy')

In [ ]:
das.trans

In [ ]:
das.set_reg_trans(power='-mtr-', poa='irr-poa-', t_amb='temp-amb-', w_vel='wind-valuesError--')

The CapData.plot method creates a group of time series plots that are useful for performing an initial visual inspection of the imported data.

The plots are structured around the translation dictionary groupings.  A single plot is generated for each different type of data (translation dictionary keys) and each column within that measurement type (translation dictionary values) is plotted as a separate series on the plot.  In this example there are two different weather stations, which each have pyranometers measuring plane of array and global horizontal irradiance. This arrangement of sensors results in two plots which each have two lines.

In [ ]:
das.plot(marker='line')

In [ ]:
sim = pvc.CapData()

In [ ]:
sim.load_data(load_pvsyst=True)

In [ ]:
sim.trans

In [ ]:
sim.drop_cols(['GlobHor'])

In [ ]:
sim.set_reg_trans(power='real_pwr--', poa='irr-ghi-', t_amb='temp-amb-', w_vel='wind--')

In [ ]:
# sim.plot()

# Performing Capacity Test

The CapTest object provides a framework that facilitates performing the steps of a capacity test following the ASTM standard. The methods of CapTest class provide convenient ways to perform various filtering steps, perform linear regressions, summarize the methods applied, calculate reporting conditions, and calculate the final results of a test.


A CapTest object must be initialized with two CapData objects: one containing measured data and one containing simulated data.  A tolerance for for the capacity test is also required.

In [ ]:
cptest = pvc.CapTest(das, sim, '+/- 5')

### Analysis of PVsyst Data

In [ ]:
# Write over cptest.flt_sim dataframe with a copy of the original unfiltered dataframe
# cptest.reset_flt('sim')

In [ ]:
cptest.scatter('sim')

In [ ]:
cptest.filter_irr('sim', 0, 900)
cptest.filter_irr('sim', 0.1, 900)
cptest.filter_irr('sim', 200, 900)

In [ ]:
cptest.summary()

In [ ]:
cptest.scatter('sim')

In [ ]:
cptest.filter_time('sim', test_date='5/15/1990', days=60)

In [ ]:
cptest.reg_cpt('sim', filter=True)

In [ ]:
cptest.filter_pvsyst('sim')

In [ ]:
cptest.sim.df.index[0]

____
#### Calculation of Reporting Conditions from PVsyst Data

In [ ]:
cptest.rep_cond('sim', test_date='5/15/1990', days=30)

----

In [ ]:
cptest.filter_irr('sim', 0.5, 1.5, ref_val=cptest.rc['poa'])

In [ ]:
cptest.reg_cpt('sim')

In [ ]:
cptest.summary()

_____

### Analysis of Measured Data

In [ ]:
cptest.reset_flt('das')

In [ ]:
cptest.filter_sensors('das', skip_strs=['wind'])

In [ ]:
cptest.filter_missing('das')

In [ ]:
cptest.summary()

In [ ]:
cptest.filter_outliers('das')

In [ ]:
# cptest.flt_das.plot(marker='line-circle')

In [ ]:
cptest.agg_sensors('das', irr='mean')

In [ ]:
cptest.scatter('das')

In [ ]:
cptest.reg_cpt('das', filter=True, summary=False)

In [ ]:
cptest.summary()

In [ ]:
cptest.scatter('das')

In [ ]:
cptest.filter_irr('das', 0.5, 1.5, ref_val=cptest.rc['poa'])

In [ ]:
cptest.reg_cpt('das')

In [ ]:
cptest.summary()

# Capacity Test Results

In [ ]:
cptest.res_summary(6000)